In [1]:
import os
import pickle
import timeit
from datetime import datetime, timedelta
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
nlct_dir_path = "/home/jturner/false_low_clouds_2023/Nighttime_Low_Cloud_Test/nlct_data/"

In [3]:
nlct_dates = sorted(os.listdir(nlct_dir_path))

#--- Removing any non-date directory items
#--- Currently only including 2023
nlct_dates = [x for x in nlct_dates if x[:6] == '202308' or x[:6] == '202309']
nlct_dates

['20230801',
 '20230802',
 '20230803',
 '20230804',
 '20230805',
 '20230806',
 '20230807',
 '20230808',
 '20230809',
 '20230810',
 '20230811',
 '20230812',
 '20230813',
 '20230814',
 '20230815',
 '20230816',
 '20230817',
 '20230818',
 '20230819',
 '20230820',
 '20230821',
 '20230822',
 '20230823',
 '20230824',
 '20230825',
 '20230826',
 '20230827',
 '20230828',
 '20230829',
 '20230830',
 '20230831',
 '20230901',
 '20230902',
 '20230903',
 '20230904',
 '20230905',
 '20230906',
 '20230907',
 '20230908',
 '20230909',
 '20230910',
 '20230911',
 '20230912',
 '20230913',
 '20230914',
 '20230915',
 '20230916',
 '20230917',
 '20230918',
 '20230919',
 '20230920',
 '20230921',
 '20230922',
 '20230923',
 '20230924',
 '20230925',
 '20230926',
 '20230927',
 '20230928',
 '20230929',
 '20230930']

Creating the static feature function: 

In [4]:
def get_static_features(BTD_c, nlct_dir_path, nlct_date):
    #--- Initialize an array of 1's in the shape of the final image
    BTD_prod = [ [1]*len(BTD_c[0][0]) for i in range(len(BTD_c[0]))]

    #--- Starting the timer
    starttime = timeit.default_timer()

    #--- Include all times or just top of hour (TOH)
    all_files = sorted(os.listdir(nlct_dir_path+nlct_date+'/'))
    toh_files = [filename for filename in all_files if filename.endswith('00M')]

    for filename in toh_files:
            #print(filename)

            #--- Open the NLCT file
            file = open(nlct_dir_path+nlct_date+'/'+filename, 'rb')
            BTD = pickle.load(file)
            file.close()

            #--- Crop to the specified region
            BTD = BTD.where((BTD.lat>min_lat) & (BTD.lat<max_lat) & (BTD.lon>min_lon) & (BTD.lon<max_lon))
            BTD = BTD.dropna(dim="y", how="all")
            BTD = BTD.dropna(dim="x", how="all")

            #--- Set negative values to zero
            BTD = np.where(BTD < 0, 0, BTD)

            #--- V1: Multiply to only enhance static clouds
            #BTD_prod = BTD_prod*BTD

            #--- V1: Limit runaway large values to 3
            #BTD_prod = np.where(BTD_prod > 3, 3, BTD_prod)
            
            #--- Mitigating error where BTD is not loaded
            if np.shape(BTD) == (1,0,0):
                BTD = [ [0]*len(BTD_c[0][0]) for i in range(len(BTD_c[0]))]
            
            #--- V2: Sum to enhance static clouds
            BTD_prod = BTD_prod+BTD


    #print("Completed: ", timeit.default_timer() - starttime)

    return BTD_prod

Creating the static value function: 

In [5]:
def get_static_values(BTD_c, nlct_dir_path, nlct_date):
    
    BTD_prod = get_static_features(BTD_c, nlct_dir_path, nlct_date)
    
    #--- Create date variable
    date = datetime.strptime(nlct_date, "%Y%m%d")
    
    #--- Get the next day in sequence
    date2 = date + timedelta(days=1)
    nlct_date2 = date2.strftime("%Y%m%d")

    #--- Run static features for the next day
    BTD_prod2 = get_static_features(BTD_c, nlct_dir_path, nlct_date2)
    
    #--- Combine current day and next day static features
    BTD_tot = BTD_prod + BTD_prod2
    
    #--- V2: Limit to top 25%
    BTD_tot_lim = np.nan_to_num(BTD_tot, nan=0)
    BTD_tot_lim = BTD_tot_lim/np.max(BTD_tot_lim)
    limit = 0.75
    BTD_tot_lim = np.where(BTD_tot_lim < limit, 0, BTD_tot_lim)
    BTD_tot_lim = np.where(BTD_tot_lim > limit, 1, BTD_tot_lim)

    #--- Convert BTD prod (2d array) to value
    static_value = np.nansum(BTD_tot_lim)
    
    return date, static_value

Create sample to use as framework for loop:

In [6]:
sample_date = '20220914'
sample_nlct_path = "/home/jturner/false_low_clouds_2023/Nighttime_Low_Cloud_Test/nlct_data/"+sample_date+"/"
sample_filename = 'goes_e_ntlc_'+sample_date[0:4]+'_'+sample_date[4:6]+'_'+sample_date[6:8]+'_00H_00M'

file = open(sample_nlct_path+sample_filename, 'rb')
BTD = pickle.load(file)
file.close()

#---Georges Bank
min_lon = -70.5
min_lat = 39
max_lon = -67
max_lat = 43

BTD_c = BTD.where((BTD.lat>min_lat) & (BTD.lat<max_lat) & (BTD.lon>min_lon) & (BTD.lon<max_lon))
BTD_c = BTD_c.dropna(dim="y", how="all")
BTD_c = BTD_c.dropna(dim="x", how="all")

Running the static value function for each date in NLCT directory: 

In [7]:
data = {'Date': [], 'StaticValue': []}
df = pd.DataFrame(data)

In [8]:
for nlct_date in tqdm(nlct_dates[:-1], desc="Processing", ncols=100):
    date, static_value = get_static_values(BTD_c, nlct_dir_path, nlct_date)
    #print(date, static_value)
    
    data_to_add = {
        'Date': date,
        'StaticValue': static_value
    }
    df = df.append(data_to_add, ignore_index=True)

Processing:  35%|█████████████████▊                                 | 21/60 [09:04<16:51, 25.94s/it]


ValueError: operands could not be broadcast together with shapes (1,144,156) (1,19,137) 

In [ ]:
df

In [ ]:
fig,ax=plt.subplots(1, figsize=(12,3))
ax.set_ylabel("Total Static Feature Value", fontsize=10)
ax.set_xlabel("Datetime", fontsize=10)
ax.set_title("Static Elements for Georges Bank", fontsize=25)

ax.plot(df.Date, df.StaticValue, c='black', linewidth=3)